In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import random
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
#from xgboost import plot_tree
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [2]:
# https://keras.io/models/sequential/

In [21]:
#pip install keras_contrib
from keras_contrib.layers import SReLU
from keras.layers import Dense, Dropout

In [3]:
data = pd.read_csv("vars_final_zscale.csv", index_col = 0)

In [4]:
# Look at the data
data.head()

,fraud_label,Days_fulladdress,fulladdress_30_count,fulladdress_7_count,ssn_30_count,Days_firstname_ssn,Days_fulladdress_homephone,fulladdresshomephone_30_count,ssnnamedob_30_count,ssn_14_count,fulladdresshomephone_14_count,namedob_14_count,firstnamessn_14_count,ssnnamedob_14_count,lastnamessn_14_count,fulladdresshomephone_7_count,homephone_14_count,homephone_1_count
record,,,,,,,,,,,,,,,,,,
19277,1,-3.230865,4.607631,5.073931,-0.099762,0.39406,-3.464852,5.758508,-0.097878,-0.072852,6.095677,-0.073782,-0.071503,-0.071056,-0.071466,6.416826,1.265044,-0.321508
19278,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,2.981941,1.269795
19279,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,-1.024153,-0.321508
19280,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,-0.451854,-0.321508
19281,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,-1.024153,-0.321508


Seperating OOT

From previous notebook we know OOT is from Nov 1 (833508)

In [6]:
# split data into out of date(after 11/1/10), train and test
oot_df = data.iloc[833508:,:]
trte_df = data.iloc[:833507,:]

In [11]:
def multipltrun_I(a=5,v=6):
    '''
    This function run model on different samples based on user input:
    "a" (int) how many randome samples, default as 5
    "v" (int) how many variables, default as 6 (most important ones from backward selection)
    users can modify model based on different machine learning algorithm and its parameters
    
    FDR is calculated by first sorting outcome in descending order and cut off at 3%,
    sum number of fraud records on top 3% and divided by total fraud racords for that sample
    
    Final output would be a dataframe contains FDR at 3% for training set, testing set, and oot. 
    '''
    #declare dict
    FDRdict={"train":[],"test":[],"oot":[]}
    
    for i in range(a):        
        #split training and testing
        train, test = train_test_split(trte_df, test_size=0.2,random_state=i)        
        
        # split lables and features and t
        train_lab = train["fraud_label"]
        train_fea = train.iloc[:,1:v+1]

        test_lab = test["fraud_label"]
        test_fea = test.iloc[:,1:v+1]

        oot_lab=oot_df["fraud_label"]
        oot_fea=oot_df.iloc[:,1:v+1]

        #define model
        #----------------
        # Initialising the ANN
        classifier = Sequential()
        # Adding the input layer and the first hidden layer
        classifier.add(Dense(units =15 , kernel_initializer = 'uniform', activation = 'relu', input_dim = v))#input_dim = 29
        # Adding the second hidden layer
        classifier.add(Dense(units = 15, kernel_initializer = 'uniform', activation = 'relu'))
        # Adding the output layer
        classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
        # Compiling the ANN
        classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
        
        
        # Fitting the ANN to the Training set
        classifier.fit(train_fea, train_lab, batch_size = 32, epochs = 5) # epochs = 100
        
    
        #calculate FDR
        for sets in ["train","test","oot"]:
            fea=vars()[sets+'_fea'] 
            lab=vars()[sets+'_lab']
            prob=pd.DataFrame(classifier.predict(fea)) #modify based on your model
            result=pd.concat([pd.DataFrame(lab).reset_index(),pd.DataFrame(prob)],axis=1)
            topRows=int(round(len(result)*0.03))
            top3per=result.sort_values(by=0,ascending=False).head(topRows)
            FDR=sum(top3per.loc[:,'fraud_label'])/sum(result.loc[:,'fraud_label'])
            FDRdict[sets].append(FDR)
    
    #convert into dataframe
    FDR_df=pd.DataFrame(FDRdict)
   
    #add new row to calculate mean
    FDR_df.loc['mean']=FDR_df.mean()
    
    return FDR_df

In [12]:
multipltrun_I()

Epoch 1/5
666805/666805 [==============================] - 126s 188us/step - loss: 0.0501 - acc: 0.9909
Epoch 2/5
666805/666805 [==============================] - 99s 148us/step - loss: 0.0462 - acc: 0.9912
Epoch 3/5
666805/666805 [==============================] - 108s 162us/step - loss: 0.0460 - acc: 0.9912
Epoch 4/5
666805/666805 [==============================] - 114s 171us/step - loss: 0.0460 - acc: 0.9912
Epoch 5/5
666805/666805 [==============================] - 123s 185us/step - loss: 0.0459 - acc: 0.9912
Epoch 1/5
666805/666805 [==============================] - 116s 173us/step - loss: 0.0513 - acc: 0.9908
Epoch 2/5
666805/666805 [==============================] - 129s 193us/step - loss: 0.0469 - acc: 0.9910
Epoch 3/5
666805/666805 [==============================] - 120s 179us/step - loss: 0.0466 - acc: 0.9911
Epoch 4/5
666805/666805 [==============================] - 116s 174us/step - loss: 0.0465 - acc: 0.9911
Epoch 5/5
666805/666805 [==============================] - 116s 1

,train,test,oot
0,0.535003,0.524003,0.525720
1,0.528676,0.541843,0.525248
2,0.532865,0.532319,0.525248
3,0.533077,0.528169,0.525248
4,0.532767,0.527933,0.525720
mean,0.532478,0.530853,0.525437


### Approach II

In [13]:
# Define a model generator
def generate_model(in_Dim,l1_Units=12,dropOut=0.2):
    _model = Sequential()
    _model.add(Dense(l1_Units, input_dim=in_Dim))
    _model.add(SReLU())
    _model.add(Dropout(dropOut))
    _model.add(Dense(1, activation='sigmoid'))
    _model.compile(loss='binary_crossentropy', optimizer='adam')
    return _model

In [15]:
trte_df.head()

,fraud_label,Days_fulladdress,fulladdress_30_count,fulladdress_7_count,ssn_30_count,Days_firstname_ssn,Days_fulladdress_homephone,fulladdresshomephone_30_count,ssnnamedob_30_count,ssn_14_count,fulladdresshomephone_14_count,namedob_14_count,firstnamessn_14_count,ssnnamedob_14_count,lastnamessn_14_count,fulladdresshomephone_7_count,homephone_14_count,homephone_1_count
record,,,,,,,,,,,,,,,,,,
19277,1,-3.230865,4.607631,5.073931,-0.099762,0.39406,-3.464852,5.758508,-0.097878,-0.072852,6.095677,-0.073782,-0.071503,-0.071056,-0.071466,6.416826,1.265044,-0.321508
19278,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,2.981941,1.269795
19279,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,-1.024153,-0.321508
19280,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,-0.451854,-0.321508
19281,0,0.407271,-0.102765,-0.067384,-0.099762,0.39406,0.379935,-0.095718,-0.097878,-0.072852,-0.070587,-0.073782,-0.071503,-0.071056,-0.071466,-0.057622,-1.024153,-0.321508


In [16]:
# Estimate class weights since the dataset is unbalanced
from sklearn.utils import compute_class_weight
class_weights = dict(zip([0, 1], compute_class_weight('balanced', [0, 1], trte_df["fraud_label"])))

In [17]:
# Define callbacks
from keras.callbacks import BaseLogger, EarlyStopping, ReduceLROnPlateau, TensorBoard
baselogger = BaseLogger()
earlystop = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5, verbose=0, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
tensor_board = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=False)

In [18]:
def multipltrun(a=5,v=6,l1_Uts = 5,dropOt=0.2):
    '''
    This function run model on different samples based on user input:
    "a" (int) how many randome samples, default as 5
    "v" (int) how many variables, default as 6 (most important ones from backward selection)
    users can modify model based on different machine learning algorithm and its parameters
    
    FDR is calculated by first sorting outcome in descending order and cut off at 3%,
    sum number of fraud records on top 3% and divided by total fraud racords for that sample
    
    Final output would be a dataframe contains FDR at 3% for training set, testing set, and oot. 
    '''
    #declare dict
    FDRdict={"train":[],"test":[],"oot":[]}
    
    for i in range(a):        
        #split training and testing
        train, test = train_test_split(trte_df, test_size=0.2,random_state=i)        
        
        # split lables and features and t
        train_lab = train["fraud_label"]
        train_fea = train.iloc[:,1:v+1]

        test_lab = test["fraud_label"]
        test_fea = test.iloc[:,1:v+1]

        oot_lab=oot_df["fraud_label"]
        oot_fea=oot_df.iloc[:,1:v+1]

        #define model
        #----------------
        # Initialising the ANN
        classifier = generate_model(v,l1_Uts,dropOt)
       
        
        # Fitting the ANN to the Training set
        classifier.fit(train_fea, train_lab, batch_size = 32, epochs = 2) # epochs = 100
        
        classifier.fit(train_fea, train_lab,
                        batch_size=1200,
                        epochs=10,
                        verbose=0,
                        shuffle=True,
                        validation_data=(test_fea, test_lab),
                        class_weight=class_weights,
                        callbacks=[baselogger, earlystop, reduce_lr, tensor_board])
    
        #calculate FDR
        for sets in ["train","test","oot"]:
            fea=vars()[sets+'_fea'] 
            lab=vars()[sets+'_lab']
            prob=pd.DataFrame(classifier.predict(fea)) #modify based on your model
            result=pd.concat([pd.DataFrame(lab).reset_index(),pd.DataFrame(prob)],axis=1)
            topRows=int(round(len(result)*0.03))
            top3per=result.sort_values(by=0,ascending=False).head(topRows)
            FDR=sum(top3per.loc[:,'fraud_label'])/sum(result.loc[:,'fraud_label'])
            FDRdict[sets].append(FDR)
    
    #convert into dataframe
    FDR_df=pd.DataFrame(FDRdict)
   
    #add new row to calculate mean
    FDR_df.loc['mean']=FDR_df.mean()
    
    return FDR_df

In [22]:
multipltrun(a=10,v=12,l1_Uts=12,dropOt=0.2)

Epoch 1/2
666805/666805 [==============================] - 139s 209us/step - loss: 0.0550
Epoch 2/2
666805/666805 [==============================] - 131s 196us/step - loss: 0.0480
Epoch 1/2
666805/666805 [==============================] - 131s 196us/step - loss: 0.0559
Epoch 2/2
666805/666805 [==============================] - 131s 196us/step - loss: 0.0485
Epoch 1/2
666805/666805 [==============================] - 132s 198us/step - loss: 0.0557
Epoch 2/2
666805/666805 [==============================] - 134s 201us/step - loss: 0.0483
Epoch 1/2
666805/666805 [==============================] - 128s 192us/step - loss: 0.0555
Epoch 2/2
666805/666805 [==============================] - 128s 191us/step - loss: 0.0479
Epoch 1/2
666805/666805 [==============================] - 131s 196us/step - loss: 0.0547
Epoch 2/2
666805/666805 [==============================] - 128s 192us/step - loss: 0.0477
Epoch 1/2
666805/666805 [==============================] - 129s 193us/step - loss: 0.0583
Epoch 2/2


,train,test,oot
0,0.535524,0.522783,0.515809
1,0.530536,0.540575,0.518641
2,0.531935,0.531897,0.513922
3,0.534427,0.528583,0.525248
4,0.531823,0.530726,0.514866
5,0.532894,0.533476,0.524776
6,0.529504,0.536565,0.515809
7,0.536066,0.521756,0.521000
8,0.534357,0.531341,0.524304
9,0.528371,0.539787,0.516281


In [20]:
multipltrun()

NameError: name 'SReLU' is not defined

In [ ]:
model.predict_proba(test_fea).flatten()